In [58]:
import pandas as pd
import numpy as np
import datetime
import os
import telepot
import requests
import selenium
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
import time
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import re

#login information
#LOGIN_URL = 'https://www.dmm.co.jp/digital/videoa/-/list/=/sort=saleranking_asc/page=1/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36','accept-language': 'zh-TW'}
downloadpath='//navdi/DEM_UserData/btorin/Desktop/fanza_data/data_rank360/'
gekodriverpath= r'//navdi/DEM_UserData/btorin/Desktop/python_code/geckodriver.exe'

In [54]:
headerss = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36',
            'cache-control': 'max-age=0',
           'cookie':'__utmc=125690133; __utmz=125690133.1663640661.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); _gcl_au=1.1.132410237.1663640661; i3_ab=d1c57b60-847e-43eb-910b-9a1572c7ef32; app_uid=Z/6QHmMpJFWIK3kXfsYGAg==; _dga=GA1.3.1836655045.1663640661; _dga_gid=GA1.3.881656899.1663640661; _cq_duid=1.1663640661.qeNQ1k6oHF9GxyA8; _cq_suid=1.1663640661.O2czJeZStgXp6q8K; age_check_done=1; _ga_G34HHM5C8N=GS1.1.1663640660.1.0.1663640662.0.0.0; _ga_SFMSWE0TVN=GS1.1.1663640660.1.0.1663640662.0.0.0; adpf_uid=THoqMpxPYndkARmn; list_condition=%7B%22digital%22%3A%7B%22limit%22%3Anull%2C%22sort%22%3A%22saleranking_asc%22%2C%22view%22%3Anull%7D%7D; LSS_SESID=A1lRXE9CCQJYQTR6d0cKEF9WAFkQOB1bTFk%2BCSQlcycjLRsodUF0aCMiIHJ9RwoQX1EOQWE1JWJtFl1YX1UEVFFVVlUGCQoKEVlYCREpYjA6N3EweyVGC1gOVQseFwhbWEEOB0dFbEINERURCBYLUlBeFhBTDQFCWFoNWAsDbQMRAEBdRwsEC0AHCFRaEVAPR19HWQNUXhJKVQpUFwVcCQsNBTlVQlATWRBbBwlHRwoAExZYQVtUChsWXUwY; _ga=GA1.3.834099324.1663640661; _gid=GA1.3.2009709900.1663643293; __utma=125690133.1836655045.1663640661.1663640661.1663643293.2; AMP_TOKEN=%24NOT_FOUND; ckcy=2; cklg=ja; __utmt=1; mbox=session#1663643292607-134217#1663648174|check#true#1663646374; __utmb=125690133.10.10.1663643293; _dc_gtm_UA-48257133-4=1; _dd_s=logs=1&id=d96def7c-b6ab-4ce2-9618-b5f1b7fb2fed&created=1663644325200&expire=1663647235815' }        
session_requests = requests.session()

In [56]:
df_data_video_all = pd.DataFrame(columns=['頁面','網址'])
for i in range(1,4,1):  # https://www.dmm.co.jp/mono/dvd/-/list/=/list_type=release/sort=ranking/page='+str(i)+'/  'https://www.dmm.co.jp/digital/videoa/-/list/=/sort=ranking/page='+str(i)+'/'
    if i==1 : LOGIN_URL ='https://www.dmm.co.jp/digital/videoa/-/list/=/sort=saleranking_asc/'
    if i>1: LOGIN_URL = 'https://www.dmm.co.jp/digital/videoa/-/list/=/sort=saleranking_asc/page='+str(i)+'/'
    web_url_rank=session_requests.request('get',url= LOGIN_URL , headers=headerss )
    soup = BeautifulSoup(web_url_rank.text, 'html.parser')
    soup_table=soup.select("div.d-item")
    soup_scrap=soup_table[0].select("p.tmb")
    df_data_video = pd.DataFrame(columns=['頁面','網址'])
    for j in range(120):
        c_index=i#c_title=soup_scrap[j].text.replace('\n', '')
        c_url=soup_scrap[j].find_all('a')[0].get('href')
        cc_data=[c_index,c_url]
        df_data_video=df_data_video.append( pd.DataFrame([cc_data],columns =['頁面','網址']) )
    df_data_video_all=df_data_video_all.append(df_data_video)

In [59]:
#會員數據
with pd.ExcelWriter(downloadpath+"/video_data_list.xlsx" ) as video_data_list: #, index=False
    df_data_video_all.to_excel(video_data_list, sheet_name="data", index=False)
    video_data_list.save()
df_data_video_all.reset_index(inplace=True, drop=True)

Calling close() on already closed file.


In [60]:
df_data_video_all = pd.read_excel(downloadpath+"/video_data_list.xlsx" )

In [63]:
df_video_content = pd.DataFrame(columns=['網址','標題','商品發售日','出演女優','屬性標籤','番號','圖片位置'])
for web_url in df_data_video_all['網址']:
    web_content_o = session_requests.request('get',url= web_url , headers=headerss )
    soup = BeautifulSoup(web_content_o.text, 'html.parser')
    if len(soup.select("table.mg-b20"))==0 : continue
    soup_title=soup.select("div.hreview")[0].text
    soup_t=soup.select("table.mg-b20")[0]
    
    web_url1=web_url
    test_data=pd.read_html(str(soup_t))[0].T
    test_data.reset_index(inplace=True, drop=True)
    test_data.columns=test_data.iloc[0,:]
    test_data=test_data[1:2]
    d_time=test_data['商品発売日：'].values[0]
    d_actor=test_data['出演者：'].values[0]
    d_label=test_data['ジャンル：'].values[0]
    d_id=test_data['品番：'].values[0]
    try:
        soup_p=soup.select('div.tx10.pd-3')[0]
        d_picture=soup_p.find_all('a')[0].get('href')
    except:
        d_picture=''
    dd_data=[web_url1,soup_title,d_time,d_actor,d_label,d_id,d_picture]
    df_video_content=df_video_content.append( pd.DataFrame([dd_data],columns =['網址','標題','商品發售日','出演女優','屬性標籤','番號','圖片位置']) )  

In [65]:
df_all_data=df_data_video_all.merge(df_video_content, on='網址', how='left')

In [66]:
#合併後資料匯出
with pd.ExcelWriter(downloadpath+"/video_data_all.xlsx" ) as video_data_all: #, index=False
    df_all_data.to_excel(video_data_all, sheet_name="data", index=False)
    video_data_all.save()

Calling close() on already closed file.


# Download Picture

In [67]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36','accept-language': 'zh-TW'}
#payload = {'name': 'torin','age': '18'}
session_requests = requests.session()

for i in range(len(df_all_data)) :
    try:
        urls=df_all_data.loc[i,'圖片位置']
        names=df_all_data.loc[i,'番號']
        web_pic = session_requests.request('get',url= urls , headers=headers )
        with open(downloadpath+'photo/'+str(names)+'.jpg', 'wb') as f:
            text = f.write(web_pic.content)
    except:
        print('fail')

In [ ]:
##登入頁面
#fp = webdriver.FirefoxProfile()
#fp.set_preference("browser.download.folderList", 2) # 0 means to download to the desktop, 1 means to download to the default "Downloads" directory, 2 means to use the directory 
#fp.set_preference("browser.helperApps.neverAsk.saveToDisk", "text/plain,text/x-csv,text/csv,application/vnd.ms-excel,application/csv,application/x-csv,text/csv,text/comma-separated-values,text/x-comma-separated-values,text/tab-separated-values,application/pdf")
#fp.set_preference("browser.download.manager.showWhenStarting",False)
#fp.set_preference("browser.helperApps.neverAsk.openFile","text/plain,text/x-csv,text/csv,application/vnd.ms-excel,application/csv,application/x-csv,text/csv,text/comma-separated-values,text/x-comma-separated-values,text/tab-separated-values,application/pdf")
#fp.set_preference("browser.helperApps.alwaysAsk.force", False)
#fp.set_preference("browser.download.manager.useWindow", False)
#fp.set_preference("browser.download.manager.focusWhenStarting", False)
#fp.set_preference("browser.helperApps.neverAsk.openFile", "")
#fp.set_preference("browser.download.manager.alertOnEXEOpen", False)
#fp.set_preference("browser.download.manager.showAlertOnComplete", False)
#fp.set_preference("browser.download.manager.closeWhenDone", True)
#fp.set_preference("browser.download.dir", downloadpath) 
#options = Options()
##options.add_argument('--headless')
#options.binary = FirefoxBinary(r'C:/Program Files/Mozilla Firefox/firefox.exe')
#browser = webdriver.Firefox(executable_path=gekodriverpath, options=options,firefox_profile = fp)
#browser.implicitly_wait(10)
#browser.maximize_window()
#browser.get(LOGIN_URL)
#
#df_data_video_all = pd.DataFrame(columns=['頁面','網址'])
#for i in range(1,4,1):  # https://www.dmm.co.jp/mono/dvd/-/list/=/list_type=release/sort=ranking/page='+str(i)+'/  'https://www.dmm.co.jp/digital/videoa/-/list/=/sort=ranking/page='+str(i)+'/'
#    LOGIN_URL = 'https://www.dmm.co.jp/mono/dvd/-/list/=/list_type=release/sort=ranking/page='+str(i)+'/'
#    browser.get(LOGIN_URL)
#    time.sleep(2)
#    soup = BeautifulSoup(browser.page_source, 'html.parser')
#    soup_table=soup.select("div.d-item")
#    soup_scrap=soup_table[0].select("p.tmb")
#    df_data_video = pd.DataFrame(columns=['頁面','網址'])
#    for j in range(120):
#        c_index=i#c_title=soup_scrap[j].text.replace('\n', '')
#        c_url=soup_scrap[j].find_all('a')[0].get('href')
#        cc_data=[c_index,c_url]
#        df_data_video=df_data_video.append( pd.DataFrame([cc_data],columns =['頁面','網址']) )
#    df_data_video_all=df_data_video_all.append(df_data_video)

In [ ]:
#headerss = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36','cache-control': 'max-age=20',
#           'cookie':'ckcy=2; cklg=ja; __utma=125690133.180804481.1655866921.1655866921.1655866921.1; __utmc=125690133; __utmz=125690133.1655866921.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); app_uid=Z/6RlGKyhikICjBew+hQAg==; AMP_TOKEN=%24NOT_FOUND; _dga=GA1.3.180804481.1655866921; _dga_gid=GA1.3.87988861.1655866922; _dc_gtm_UA-48257133-4=1; i3_ab=824205dd-4b57-475b-925c-e24876644ce0; age_check_done=1; dmm_service=BFsBAx1FWwQCR1JXXlsJWUNeVwwBAx9KWFYNREEKRURHWkMDUQxDQFkLWlFdUUQMHBg_; mbox=check#true#1655866987|session#1655866921246-760041#1655868787; __utmt=1; __utmb=125690133.1.10.1655866921; _ga=GA1.3.180804481.1655866921; _gid=GA1.3.1871223578.1655866926; adpf_uid=nPWoGSRXhbPpTKKg; _dd_s=logs=1&id=73a21d40-46d1-437b-ba93-b5901a516f0c&created=1655866926970&expire=1655867826978' }        
#session_requests = requests.session()